In [ ]:
# deps
!pip install opencv-python easyocr numpy matplotlib

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import easyocr

# === 1) Load and preprocess ===
img = cv2.imread("pvi_cv08_spz.png")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (5, 5), 0)

# === 2) Threshold ===
thresh = cv2.adaptiveThreshold(
    blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
    cv2.THRESH_BINARY_INV, 41, 15
)

# === 3) Find the plate contour ===
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = sorted(contours, key=cv2.contourArea, reverse=True)

plate_cnt = None
for c in contours:
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)
    if len(approx) == 4:
        plate_cnt = approx
        break

if plate_cnt is None:
    raise ValueError("License plate contour not found!")

# === 4) Compute rotation to make plate vertical ===
rect = cv2.minAreaRect(plate_cnt)
angle = rect[-1]
angle = angle - 2

print(angle)

# minAreaRect conventons
#if rect[1][0] > rect[1][1]:
#    angle = 90 + angle

(h, w) = img.shape[:2]
center = (w // 2, h // 2)import cv2
import numpy as np
import matplotlib.pyplot as plt
import easyocr

# === 1) Load and preprocess ===
img = cv2.imread("pvi_cv08_spz.png")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (5, 5), 0)

# === 2) Threshold ===
thresh = cv2.adaptiveThreshold(
    blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
    cv2.THRESH_BINARY_INV, 41, 15
)

# === 3) Find the plate contour ===
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = sorted(contours, key=cv2.contourArea, reverse=True)

plate_cnt = None
for c in contours:
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)
    if len(approx) == 4:
        plate_cnt = approx
        break

if plate_cnt is None:
    raise ValueError("License plate contour not found!")

# === 4) Compute rotation to make plate vertical ===
rect = cv2.minAreaRect(plate_cnt)
angle = rect[-1]
angle = angle - 2

print(angle)

# minAreaRect conventons
#if rect[1][0] > rect[1][1]:
#    angle = 90 + angle

(h, w) = img.shape[:2]
center = (w // 2, h // 2)
M = cv2.getRotationMatrix2D(center, angle, 1.0)
rotated = cv2.warpAffine(img, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

# === 5) Crop the plate region after rotation ===
gray_rot = cv2.cvtColor(rotated, cv2.COLOR_BGR2GRAY)
thresh_rot = cv2.adaptiveThreshold(
    gray_rot, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
    cv2.THRESH_BINARY_INV, 41, 15
)

contours, _ = cv2.findContours(thresh_rot, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = sorted(contours, key=cv2.contourArea, reverse=True)

plate_box = None
for c in contours:
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)
    if len(approx) == 4:
        plate_box = cv2.boundingRect(approx)
        break

if plate_box is None:
    raise ValueError("Plate not found after rotation!")

x, y, w_box, h_box = plate_box
plate_crop = rotated[y:y + h_box, x:x + w_box]

# === 6) Rotate plate 90° clockwise for OCR ===
plate_for_ocr = cv2.rotate(plate_crop, cv2.ROTATE_90_CLOCKWISE)

# === 7) OCR ===
reader = easyocr.Reader(['en'])
result = reader.readtext(plate_for_ocr)
text_result = " ".join([r[1] for r in result])
# rotate so it becomes vertical

# === 8) Draw result ===
output = rotated.copy()
cv2.putText(output, text_result, (x, y - 20), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)
cv2.rectangle(output, (x, y), (x + w_box, y + h_box), (0, 255, 0), 2)

# === 9) Visualization ===
plt.figure(figsize=(12, 10))
plt.subplot(2, 2, 1)
plt.title("Original Image")
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

plt.subplot(2, 2, 2)
plt.title("Threshold")
plt.imshow(thresh, cmap='gray')

plt.subplot(2, 2, 3)
plt.title("Vertical Plate (for display)")
plt.imshow(cv2.cvtColor(rotated, cv2.COLOR_BGR2RGB))

plt.subplot(2, 2, 4)
plt.title(f"OCR Result: {text_result}")
plt.imshow(cv2.cvtColor(plate_for_ocr, cv2.COLOR_BGR2RGB))
plt.show()

print("Recognized text:", text_result)

M = cv2.getRotationMatrix2D(center, angle, 1.0)
rotated = cv2.warpAffine(img, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

# === 5) Crop the plate region after rotation ===
gray_rot = cv2.cvtColor(rotated, cv2.COLOR_BGR2GRAY)
thresh_rot = cv2.adaptiveThreshold(
    gray_rot, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
    cv2.THRESH_BINARY_INV, 41, 15
)

contours, _ = cv2.findContours(thresh_rot, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = sorted(contours, key=cv2.contourArea, reverse=True)

plate_box = None
for c in contours:
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)
    if len(approx) == 4:
        plate_box = cv2.boundingRect(approx)
        break

if plate_box is None:
    raise ValueError("Plate not found after rotation!")

x, y, w_box, h_box = plate_box
plate_crop = rotated[y:y + h_box, x:x + w_box]

# === 6) Rotate plate 90° clockwise for OCR ===
plate_for_ocr = cv2.rotate(plate_crop, cv2.ROTATE_90_CLOCKWISE)

# === 7) OCR ===
reader = easyocr.Reader(['en'])
result = reader.readtext(plate_for_ocr)
text_result = " ".join([r[1] for r in result])
# rotate so it becomes vertical

# === 8) Draw result ===
output = rotated.copy()
cv2.putText(output, text_result, (x, y - 20), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)
cv2.rectangle(output, (x, y), (x + w_box, y + h_box), (0, 255, 0), 2)

# === 9) Visualization ===
plt.figure(figsize=(12, 10))
plt.subplot(2, 2, 1)
plt.title("Original Image")
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

plt.subplot(2, 2, 2)
plt.title("Threshold")
plt.imshow(thresh, cmap='gray')

plt.subplot(2, 2, 3)
plt.title("Vertical Plate (for display)")
plt.imshow(cv2.cvtColor(rotated, cv2.COLOR_BGR2RGB))

plt.subplot(2, 2, 4)
plt.title(f"OCR Result: {text_result}")
plt.imshow(cv2.cvtColor(plate_for_ocr, cv2.COLOR_BGR2RGB))
plt.show()

print("Recognized text:", text_result)

In [ ]:
box = cv2.boxPoints(rect)
box = np.int0(box)

img_with_rect = img.copy()
cv2.drawContours(img_with_rect, [box], 0, (0, 0, 255), 3)  # red rectangle


center = (int(rect[0][0]), int(rect[0][1]))
cv2.circle(img_with_rect, center, 5, (255, 0, 0), -1)      # blue center dot

plt.figure(figsize=(6, 8))
plt.title("minAreaRect")
plt.imshow(cv2.cvtColor(img_with_rect, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()

In [ ]:
plt.imshow(morph, cmap='jet')
plt.colorbar()
# binary segmented image after adaptive threshold + morphological close.

In [ ]:
plt.imshow(morph_result, cmap='jet')
plt.colorbar()
# The cleaned binary mask — after extra morphological dilation/closing to leave only the plate rectangle (text removed).


In [ ]:
# convert to RGB for easyocr
# originalni postup
plt.imshow(cv2.cvtColor(img_harris, cv2.COLOR_BGR2RGB))
plt.colorbar()
# The original image with Harris corners drawn (from dst = cv2.cornerHarris(...)).

In [ ]:
plt.imshow(dst, cmap='jet'); plt.colorbar()


In [ ]:
plt.imshow(thresh, cmap='jet'); plt.colorbar()


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import easyocr

# === 1) Load and preprocess ===
img = cv2.imread("pvi_cv08_spz.png")
if img is None:
    raise FileNotFoundError("Image not found")

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (5, 5), 0)

# === 2) Threshold ===
thresh = cv2.adaptiveThreshold(
    blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
    cv2.THRESH_BINARY_INV, 41, 15
)

# === 3) Find the plate contour (same as your original) ===
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = sorted(contours, key=cv2.contourArea, reverse=True)

plate_cnt = None
for c in contours:
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)
    if len(approx) == 4:
        plate_cnt = approx
        break

if plate_cnt is None:
    raise ValueError("License plate contour not found!")

# === 4) Use Harris corners INSIDE the plate to get rotation ===
# Get ROI from the contour
x0, y0, w0, h0 = cv2.boundingRect(plate_cnt)
plate_roi_gray = gray[y0:y0 + h0, x0:x0 + w0]

# Harris on ROI
roi_float = np.float32(plate_roi_gray)
harris = cv2.cornerHarris(roi_float, blockSize=2, ksize=3, k=0.04)
harris = cv2.dilate(harris, None)

# Take only strong responses
mask = harris > 0.01 * harris.max()
ys, xs = np.where(mask)

if len(xs) < 4:
    raise ValueError("Not enough Harris corners in plate ROI")

# Build point set in GLOBAL coordinates for minAreaRect
pts = np.float32([[x0 + x, y0 + y] for x, y in zip(xs, ys)]).reshape(-1, 1, 2)

# minAreaRect on Harris points (this replaces minAreaRect on contour)
rect = cv2.minAreaRect(pts)
angle = rect[-1]

# Keep your original correction so result matches your old code
#angle = angle - 2
print("Harris-based/minAreaRect angle:", angle)

# === 5) Rotate whole image ===
(h, w) = img.shape[:2]
center = (w // 2, h // 2)
M = cv2.getRotationMatrix2D(center, angle, 1.0)
rotated = cv2.warpAffine(
    img, M, (w, h),
    flags=cv2.INTER_CUBIC,
    borderMode=cv2.BORDER_REPLICATE
)

# === 6) Crop the plate region after rotation (same logic as before) ===
gray_rot = cv2.cvtColor(rotated, cv2.COLOR_BGR2GRAY)
thresh_rot = cv2.adaptiveThreshold(
    gray_rot, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
    cv2.THRESH_BINARY_INV, 41, 15
)

contours, _ = cv2.findContours(thresh_rot, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = sorted(contours, key=cv2.contourArea, reverse=True)

plate_box = None
for c in contours:
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)
    if len(approx) == 4:
        plate_box = cv2.boundingRect(approx)
        break

if plate_box is None:
    raise ValueError("Plate not found after rotation!")

x, y, w_box, h_box = plate_box
plate_crop = rotated[y:y + h_box, x:x + w_box]

# === 7) Rotate plate 90° clockwise for OCR (same as your original) ===
plate_for_ocr = cv2.rotate(plate_crop, cv2.ROTATE_90_CLOCKWISE)

# === 8) OCR ===
reader = easyocr.Reader(['en'])
result = reader.readtext(plate_for_ocr)
text_result = " ".join([r[1] for r in result])

# === 9) Draw result ===
output = rotated.copy()
cv2.putText(output, text_result, (x, y - 20),
            cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)
cv2.rectangle(output, (x, y), (x + w_box, y + h_box), (0, 255, 0), 2)

# === 10) Visualization ===
plt.figure(figsize=(12, 10))

plt.subplot(2, 2, 1)
plt.title("Original Image")
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis('off')

plt.subplot(2, 2, 2)
plt.title("Threshold")
plt.imshow(thresh, cmap='gray')
plt.axis('off')

plt.subplot(2, 2, 3)
plt.title("Vertical Plate (for display)")
plt.imshow(cv2.cvtColor(rotated, cv2.COLOR_BGR2RGB))
plt.axis('off')

plt.subplot(2, 2, 4)
plt.title(f"OCR Result: {text_result}")
plt.imshow(cv2.cvtColor(plate_for_ocr, cv2.COLOR_BGR2RGB))
plt.axis('off')

plt.tight_layout()
plt.show()

print("Recognized text:", text_result)
